In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import AgglomerativeClustering
%matplotlib inline

In [3]:
def outlier_detection(df, ft):
    df.boxplot(column=[ft])
    plt.grid(False)
    plt.show()

In [4]:
def outliers(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR 
    upper_bound = Q3 + 1.5 * IQR
    filt = (lower_bound>df[ft]) | (upper_bound<df[ft])
    ls = df[filt].index
    return ls

In [5]:
def remove(df, ls):
    ls = list(set(ls))
    df_clean = df.drop(index=ls)
    return df_clean 

In [56]:
def calculate_classification_data(df, split_time, is_pca_on, pca_n_components, cluster_n):
    df_clean = data_cleaning(df).copy()
    if is_pca_on:
        #pca transform
        pca_data = df_clean.copy()
        pca_data_standard = scale(pca_data)
        pca_model = PCA(n_components=pca_n_components, random_state=2021)
        pca_model.fit(pca_data_standard)
        for i in range(1,pca_n_components+1):
            pca_data[f'pca_{i}'] = pca_model.transform(pca_data_standard)[:,i-1]
        data_processed = pca_data.loc[:, 'pca_1':f'pca_{pca_n_components}'].copy()
    else:
        data_processed = pd.DataFrame(scale(df_clean), index=df_clean.index, columns=df_clean.columns).copy()
    #clustering
    df_train = data_processed[data_processed.index<pd.to_datetime(split_time)].copy()
    df_test = data_processed[(pd.to_datetime(split_time)<=data_processed.index) & (data_processed.index<=pd.to_datetime(split_time)+pd.DateOffset(years=1))].copy()
        
    km = KMeans(n_clusters=cluster_n, random_state=2021)
    results = km.fit_predict(df_train)
    #calculate the Silhouetter Score
    score = metrics.silhouette_score(df_train, km.labels_, metric='euclidean')
    print(f'Silhouette Score: {score}')
    df_train['label'] = results
    df_train['label'] = df_train['label'].shift(-1)
    df_train.dropna(inplace=True)
    
    df_train.to_csv(f'train_{split_time[0:4]}.csv')
    df_test.to_csv(f'test_{split_time[0:4]}.csv')
    return df_train, df_test
    
        

In [55]:
def data_cleaning(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    df = df.apply(lambda x: x.pct_change())
    df.dropna(inplace=True)
    outlier_list = []
    for col in df.columns:
        outlier_list.extend(outliers(df, col))
    df_clean = remove(df, outlier_list)
    return df_clean

In [53]:
df = pd.read_csv('Data/rawData_Feb11.csv')
df

,Date,RAY,VIX,QQQ,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,VXUS,EEM
0,1990-01-02,200.241,17.24,NaN,0.11,NaN,NaN,NaN,NaN,NaN,1.2146,NaN,NaN
1,1990-01-03,200.116,18.19,NaN,0.10,NaN,NaN,NaN,NaN,NaN,1.2096,NaN,NaN
2,1990-01-04,198.646,19.22,NaN,0.14,NaN,NaN,NaN,NaN,NaN,1.2287,NaN,NaN
3,1990-01-05,196.850,20.11,NaN,0.20,NaN,NaN,NaN,NaN,NaN,1.2358,NaN,NaN
4,1990-01-08,197.525,20.26,NaN,0.23,NaN,NaN,NaN,NaN,NaN,1.2452,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8374,2022-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.970,48.56
8375,2022-02-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.330,49.03
8376,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.350,49.80
8377,2022-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.630,49.47


In [57]:
train_2018, test_2018 = calculate_classification_data(df, '2018-01-01', True, 8, 2)
train_2019, test_2019 = calculate_classification_data(df, '2019-01-01', True, 8, 2)
train_2020, test_2020 = calculate_classification_data(df, '2020-01-01', True, 8, 2)
train_2021, test_2021 = calculate_classification_data(df, '2021-01-01', True, 8, 2)

Silhouette Score: 0.20729856954417475
Silhouette Score: 0.20913839989081395
Silhouette Score: 0.2065296912651567
Silhouette Score: 0.20552716378840852


In [58]:
train_2018

,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,label
Date,,,,,,,,,
2011-01-31,-3.096733,0.326651,-1.143437,0.743807,-1.356938,-1.225331,-1.372614,0.006825,1.0
2011-02-01,-5.687933,0.979683,0.228141,2.145068,1.464628,-0.307035,1.135861,0.261640,0.0
2011-02-02,1.297446,-0.898246,-0.416135,-0.035624,-0.037442,-0.145237,-0.507297,0.554426,1.0
2011-02-04,-0.140739,-0.659326,-0.821436,-0.167618,2.027617,0.079125,0.561497,0.432729,1.0
2011-02-07,-0.633257,0.004004,1.266020,0.245032,0.851187,-0.004046,-0.564146,-0.331825,1.0
...,...,...,...,...,...,...,...,...,...
2017-12-20,-0.058664,1.081700,-1.870860,0.323767,0.282655,-0.108824,0.563019,0.711966,0.0
2017-12-21,0.519266,0.065336,-1.978049,0.320959,0.461508,0.121902,1.551587,0.382058,1.0
2017-12-22,-0.522514,1.215959,-0.362858,-0.232579,0.566702,0.185977,-0.815609,-1.214983,1.0


In [36]:
train_2018.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean -0.006247  0.161800
      std   2.015950  2.139783
pca_2 mean -0.027548 -0.017620
      std   1.446350  1.427405
pca_3 mean  0.019237  0.004581
      std   1.057410  0.993115
pca_4 mean  0.065605  0.023144
      std   0.958674  0.944891
pca_5 mean  0.038216 -0.017368
      std   0.826868  0.880094
pca_6 mean  0.004751  0.013639
      std   0.806717  0.798443
pca_7 mean  0.000397 -0.004420
      std   0.719184  0.709912
pca_8 mean  0.010808 -0.061784
      std   0.661341  0.686767

In [37]:
train_2019.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.101101  0.004082
      std   2.132772  2.045445
pca_2 mean -0.026515 -0.035622
      std   1.411292  1.437818
pca_3 mean  0.006772  0.004501
      std   1.017987  1.074369
pca_4 mean  0.023457  0.045789
      std   0.941696  0.951505
pca_5 mean -0.022429  0.027400
      std   0.888139  0.832009
pca_6 mean  0.009709  0.021880
      std   0.817773  0.804553
pca_7 mean  0.001243  0.014802
      std   0.727812  0.758405
pca_8 mean -0.042481  0.011679
      std   0.694154  0.658690

In [38]:
train_2020.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.089637  0.002619
      std   2.117335  2.030744
pca_2 mean -0.020301 -0.037553
      std   1.404899  1.426434
pca_3 mean  0.003569  0.020551
      std   1.039725  1.088490
pca_4 mean  0.005504  0.040554
      std   0.936952  0.949386
pca_5 mean -0.019440  0.025084
      std   0.883818  0.833288
pca_6 mean  0.007907  0.014933
      std   0.821588  0.814081
pca_7 mean -0.001835  0.014721
      std   0.742515  0.768959
pca_8 mean -0.039503  0.007276
      std   0.693687  0.655838

In [39]:
train_2021.groupby('label').agg(['mean', 'std']).transpose()

label            0.0       1.0
pca_1 mean  0.068618 -0.048819
      std   2.143774  2.027236
pca_2 mean  0.026032 -0.049989
      std   1.397406  1.426464
pca_3 mean -0.000414  0.011116
      std   1.087306  1.095950
pca_4 mean -0.009574  0.044486
      std   0.939768  0.942816
pca_5 mean -0.015625  0.013196
      std   0.893176  0.857122
pca_6 mean -0.005993  0.013042
      std   0.838490  0.815781
pca_7 mean  0.019051  0.012016
      std   0.798136  0.782206
pca_8 mean -0.025330  0.009359
      std   0.715721  0.663529

#### cluster the whole dataset

In [23]:
df_clean

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD
Date,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,-0.004184,-0.008817,-0.009416
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,-0.010587,0.007602,0.010871
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,-0.007974,-0.017618,-0.009888
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,-0.020095,-0.020752,-0.011316
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,0.016703,-0.000727,-0.005160
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-10,-0.001461,0.034115,-0.003645,-0.001406,0.000658,-0.007399,-0.006024,-0.008492,-0.013152,0.000779,0.004489,-0.005764,-0.002993
2022-01-11,0.009908,-0.051031,0.011454,0.019105,0.015022,0.014274,-0.006061,0.038221,0.017693,0.010952,0.013957,0.001540,0.003620
2022-01-14,0.000467,-0.055145,0.000407,-0.014383,0.006223,-0.000935,0.044304,0.020701,-0.027607,-0.002690,-0.005457,-0.009052,-0.003841


In [29]:
km = KMeans(n_clusters=3, random_state=2021)
results_whole = km.fit_predict(df_clean)
#calculate the Silhouetter Score
score = metrics.silhouette_score(df_clean, km.labels_, metric='euclidean')
print(f'Silhouette Score: {score}')

Silhouette Score: 0.2496869919356694


In [30]:
df_test_whole = df_clean.copy()
df_test_whole['label'] = results_whole-1

In [31]:
df_test_whole.to_csv('benchmark_data_3_cluster.csv')

In [32]:
df_test_whole.head()

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD,label
Date,,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,-0.004184,-0.008817,-0.009416,-1
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,-0.010587,0.007602,0.010871,0
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,-0.007974,-0.017618,-0.009888,0
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,-0.020095,-0.020752,-0.011316,1
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,0.016703,-0.000727,-0.005160,0


#### find expected return and vol for each group

In [33]:

df_test_whole.groupby('label').agg(['mean', 'std']).transpose()

label                -1         0         1
RAY      mean -0.004872  0.001254  0.007005
         std   0.004299  0.003702  0.004813
VIX      mean  0.068080 -0.005807 -0.074200
         std   0.029603  0.019636  0.028418
IWV      mean -0.004897  0.001255  0.006989
         std   0.004318  0.003732  0.004838
EEM      mean -0.002378  0.001042  0.003122
         std   0.009774  0.009660  0.009642
QQQ      mean -0.005135  0.001982  0.007815
         std   0.006355  0.005831  0.006958
VXUS     mean -0.004633  0.000944  0.006007
         std   0.005290  0.005255  0.005615
T10Y3M   mean -0.007835 -0.002782  0.010282
         std   0.034408  0.031933  0.032450
CL1      mean -0.004897  0.001225  0.008320
         std   0.017619  0.017053  0.019203
HG1      mean -0.001845  0.000894  0.003025
         std   0.010279  0.010700  0.011709
GC1      mean  0.000615  0.000410 -0.000072
         std   0.006620  0.006271  0.006245
XAG      mean -0.001106  0.000764  0.001798
         std   0.011231  0.010267  0.010898
VNQ      mean -0.002816  0.000587  0.004860
         std   0.008259  0.007522  0.008222
EURtoUSD mean -0.000048  0.000113  0.000053
         std   0.003795  0.004146  0.004302

In [ ]:
#based on the above results, 1: bull market, -1: bear market 0: stable market

In [39]:
#Hierarchical Clustering
hr_cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
hr_cluster.fit_predict(df_clean)
hr_score = metrics.silhouette_score(df_clean, hr_cluster.labels_, metric='euclidean')
print(f'Silhouette Score: {hr_score}')

Silhouette Score: 0.22456252552673217
